In [51]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt 
import seaborn as sns

In [52]:
mydir = '/Users/supriyajadhav/Documents/MS422-Practical-Machine-Learning/Project422/'
processed_df = pd.read_csv(mydir + 'preprocessed_train_data.csv')

/var/folders/r1/dsylm7cn0vg3015hchpddz9m0000gn/T/ipykernel_54787/3622642928.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  processed_df = pd.read_csv(mydir + 'preprocessed_train_data.csv')


In [53]:
processed_df[processed_df['sales'] != 0.0].head()

,Unnamed: 0,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,holiday_type,IS_HOLIDAY,oil_price,month,year,week_of_day,week_of_month,week_number
563,563,563,2013-01-01,25,BEAUTY,2.000,0,Salinas,Santa Elena,D,1,Holiday,1.0,93.14,1,2013,2,1,1
564,564,564,2013-01-01,25,BEVERAGES,810.000,0,Salinas,Santa Elena,D,1,Holiday,1.0,93.14,1,2013,2,1,1
566,566,566,2013-01-01,25,BREAD/BAKERY,180.589,0,Salinas,Santa Elena,D,1,Holiday,1.0,93.14,1,2013,2,1,1
568,568,568,2013-01-01,25,CLEANING,186.000,0,Salinas,Santa Elena,D,1,Holiday,1.0,93.14,1,2013,2,1,1
569,569,569,2013-01-01,25,DAIRY,143.000,0,Salinas,Santa Elena,D,1,Holiday,1.0,93.14,1,2013,2,1,1


In [54]:
processed_df['date'] = pd.to_datetime(processed_df['date'])
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008280 entries, 0 to 3008279
Data columns (total 19 columns):
 #   Column         Dtype         
---  ------         -----         
 0   Unnamed: 0     int64         
 1   id             int64         
 2   date           datetime64[ns]
 3   store_nbr      int64         
 4   family         object        
 5   sales          float64       
 6   onpromotion    int64         
 7   city           object        
 8   state          object        
 9   type           object        
 10  cluster        int64         
 11  holiday_type   object        
 12  IS_HOLIDAY     float64       
 13  oil_price      float64       
 14  month          int64         
 15  year           int64         
 16  week_of_day    int64         
 17  week_of_month  int64         
 18  week_number    int64         
dtypes: datetime64[ns](1), float64(3), int64(10), object(5)
memory usage: 436.1+ MB


In [55]:
unique_store_family_df = processed_df[['store_nbr', 'family']].drop_duplicates()

# Reset index if you want a clean DataFrame
unique_store_family_df = unique_store_family_df.reset_index(drop=True)

# Display the resulting DataFrame
print(unique_store_family_df.head())
print(len(unique_store_family_df))

   store_nbr      family
0          1  AUTOMOTIVE
1          1   BABY CARE
2          1      BEAUTY
3          1   BEVERAGES
4          1       BOOKS
1782


In [56]:
missing_data = processed_df.isnull().sum()
print(missing_data)


Unnamed: 0             0
id                     0
date                   0
store_nbr              0
family                 0
sales                  0
onpromotion            0
city                   0
state                  0
type                   0
cluster                0
holiday_type     2746128
IS_HOLIDAY             0
oil_price         862752
month                  0
year                   0
week_of_day            0
week_of_month          0
week_number            0
dtype: int64


In [57]:
processed_df = processed_df.drop(columns=['holiday_type','oil_price'])

In [58]:
# Step 1: Group by 'store_nbr' and 'family'
grouped = processed_df.groupby(['store_nbr', 'family'])

# Step 2: Sort each group by 'date' in ascending order
sorted_df = grouped.apply(lambda x: x.sort_values('date')).reset_index(drop=True)

# Optional: If you want to create a dictionary of DataFrames, one for each group
grouped_dict = {f"processed_{store}_{family}": group.sort_values('date') 
                for (store, family), group in grouped}

# Display the result
sorted_df.head()


,Unnamed: 0,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,IS_HOLIDAY,month,year,week_of_day,week_of_month,week_number
0,0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,1.0,1,2013,2,1,1
1,1782,1782,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13,0.0,1,2013,3,1,1
2,3564,3564,2013-01-03,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13,0.0,1,2013,4,1,1
3,5346,5346,2013-01-04,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13,0.0,1,2013,5,1,1
4,7128,7128,2013-01-05,1,AUTOMOTIVE,5.0,0,Quito,Pichincha,D,13,1.0,1,2013,6,1,1


In [59]:
processed_1_AUTOMOTIVE_df = grouped_dict['processed_1_BEVERAGES']

In [60]:
print(processed_1_AUTOMOTIVE_df.columns)

Index(['Unnamed: 0', 'id', 'date', 'store_nbr', 'family', 'sales',
       'onpromotion', 'city', 'state', 'type', 'cluster', 'IS_HOLIDAY',
       'month', 'year', 'week_of_day', 'week_of_month', 'week_number'],
      dtype='object')


In [61]:
processed_1_AUTOMOTIVE_df.shape

(1688, 17)

In [62]:
# Extract date components
processed_1_AUTOMOTIVE_df['day'] = processed_1_AUTOMOTIVE_df['date'].dt.day

# Create cyclical features
processed_1_AUTOMOTIVE_df['month_sin'] = np.sin(2 * np.pi * processed_1_AUTOMOTIVE_df['month'] / 12)
processed_1_AUTOMOTIVE_df['month_cos'] = np.cos(2 * np.pi * processed_1_AUTOMOTIVE_df['month'] / 12)
processed_1_AUTOMOTIVE_df['weekday_sin'] = np.sin(2 * np.pi * processed_1_AUTOMOTIVE_df['week_of_day'] / 7)
processed_1_AUTOMOTIVE_df['weekday_cos'] = np.cos(2 * np.pi * processed_1_AUTOMOTIVE_df['week_of_day'] / 7)

# Optionally, create lag features and rolling statistics
# processed_1_AUTOMOTIVE_df['sales_lag_1'] = processed_1_AUTOMOTIVE_df['sales'].shift(1)
# processed_1_AUTOMOTIVE_df['sales_rolling_mean_7'] = processed_1_AUTOMOTIVE_df['sales'].rolling(window=7).mean()

# Drop rows with NaN values if needed
processed_1_AUTOMOTIVE_df = processed_1_AUTOMOTIVE_df.dropna()

In [63]:
processed_1_AUTOMOTIVE_df.shape

(1688, 22)

In [64]:
# drop unnecessary columns
processed_1_AUTOMOTIVE_df = processed_1_AUTOMOTIVE_df.drop([
    'Unnamed: 0',
    'date',
    'id',
    'city',
    'state',
    'store_nbr',
    'family',
    'type' ], 
    axis = 1)

In [65]:
print(processed_1_AUTOMOTIVE_df.columns)

Index(['sales', 'onpromotion', 'cluster', 'IS_HOLIDAY', 'month', 'year',
       'week_of_day', 'week_of_month', 'week_number', 'day', 'month_sin',
       'month_cos', 'weekday_sin', 'weekday_cos'],
      dtype='object')


In [66]:
processed_1_AUTOMOTIVE_df = processed_1_AUTOMOTIVE_df.astype('float64')
processed_1_AUTOMOTIVE_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1688 entries, 3 to 3006501
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sales          1688 non-null   float64
 1   onpromotion    1688 non-null   float64
 2   cluster        1688 non-null   float64
 3   IS_HOLIDAY     1688 non-null   float64
 4   month          1688 non-null   float64
 5   year           1688 non-null   float64
 6   week_of_day    1688 non-null   float64
 7   week_of_month  1688 non-null   float64
 8   week_number    1688 non-null   float64
 9   day            1688 non-null   float64
 10  month_sin      1688 non-null   float64
 11  month_cos      1688 non-null   float64
 12  weekday_sin    1688 non-null   float64
 13  weekday_cos    1688 non-null   float64
dtypes: float64(14)
memory usage: 197.8 KB


In [67]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Selecting features
features = ['onpromotion', 'cluster', 'IS_HOLIDAY', 'month', 'year',
       'week_of_day', 'week_of_month', 'week_number', 'day', 'month_sin',
       'month_cos', 'weekday_sin', 'weekday_cos'
       #, 'sales_lag_1',
       #'sales_rolling_mean_7'
       ]
target = 'sales'

# Feature scaling
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Get the remaining records for the training set
X = processed_1_AUTOMOTIVE_df[features]
y = processed_1_AUTOMOTIVE_df[target]

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train_scaled = feature_scaler.fit_transform(X_train)
X_test_scaled = feature_scaler.fit_transform(X_test)
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1,1))
y_test_scaled = target_scaler.fit_transform(y_test.values.reshape(-1,1))


In [68]:
print(X_train_scaled.shape)

(1350, 13)


In [69]:
print(y_train_scaled.shape)

(1350, 1)


In [70]:
print(X_test_scaled.shape)

(338, 13)


In [71]:
print(y_test_scaled.shape)

(338, 1)


## LSTM Model

In [72]:
import tensorflow as tf

def rmsle(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    return tf.sqrt(tf.reduce_mean(tf.square(tf.math.log(y_pred + 1) - tf.math.log(y_true + 1))))


In [73]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Reshape data to fit LSTM input shape (num_samples, timesteps, num_features)
timesteps = 1  # You need to define this based on your data and model requirements
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

# Building the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(units=60, return_sequences=True, input_shape=(timesteps, X_train_reshaped.shape[2])))
lstm_model.add(Dropout(0.5))
lstm_model.add(LSTM(units=60, return_sequences=False))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(units=1))

lstm_model.compile(optimizer='adam', loss=rmsle)
#lstm_model.summary()

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the LSTM model
lstm_model.fit(X_train_reshaped, y_train_scaled, epochs=50, batch_size=32, 
               validation_data=(X_test_reshaped, y_test_scaled),
               callbacks=[early_stopping])


In [74]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dropout, Dense
# from scikeras.wrappers import KerasRegressor
# from sklearn.model_selection import GridSearchCV

# # Define the function to create the LSTM model
# def create_lstm_model(units=50, dropout_rate=0.5, optimizer='adam'):
#     model = Sequential()
#     model.add(LSTM(units=units, return_sequences=True, input_shape=(timesteps, X_train_reshaped.shape[2])))
#     model.add(Dropout(dropout_rate))
#     model.add(LSTM(units=units, return_sequences=False))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(units=1))
#     model.compile(optimizer=optimizer, loss=rmsle)
#     return model

# # Define the grid of hyperparameters to search, using lambda functions to pass parameters
# param_grid = {
#     'build_fn': [
#         lambda: create_lstm_model(units=50, dropout_rate=0.2, optimizer='adam'),
#         lambda: create_lstm_model(units=50, dropout_rate=0.5, optimizer='adam'),
#         lambda: create_lstm_model(units=50, dropout_rate=0.3, optimizer='adam'),
#         lambda: create_lstm_model(units=60, dropout_rate=0.2, optimizer='adam'),
#         lambda: create_lstm_model(units=60, dropout_rate=0.5, optimizer='adam'),
#         lambda: create_lstm_model(units=60, dropout_rate=0.3, optimizer='adam'),
#         lambda: create_lstm_model(units=80, dropout_rate=0.2, optimizer='adam'),
#         lambda: create_lstm_model(units=80, dropout_rate=0.3, optimizer='adam'),
#         lambda: create_lstm_model(units=80, dropout_rate=0.5, optimizer='adam'),
#         lambda: create_lstm_model(units=50, dropout_rate=0.2, optimizer='rmsprop'),
#         lambda: create_lstm_model(units=50, dropout_rate=0.5, optimizer='rmsprop'),
#         lambda: create_lstm_model(units=50, dropout_rate=0.3, optimizer='rmsprop'),
#         lambda: create_lstm_model(units=60, dropout_rate=0.2, optimizer='rmsprop'),
#         lambda: create_lstm_model(units=60, dropout_rate=0.5, optimizer='rmsprop'),
#         lambda: create_lstm_model(units=60, dropout_rate=0.3, optimizer='rmsprop'),
#         lambda: create_lstm_model(units=80, dropout_rate=0.2, optimizer='rmsprop'),
#         lambda: create_lstm_model(units=80, dropout_rate=0.3, optimizer='rmsprop'),
#         lambda: create_lstm_model(units=80, dropout_rate=0.5, optimizer='rmsprop')
#     ],
#     'batch_size': [16, 32, 64],  # Batch size
#     'epochs': [30, 50, 100],  # Number of epochs
# }

# # Create a GridSearchCV object
# grid_search = GridSearchCV(estimator=KerasRegressor(build_fn=create_lstm_model, verbose=0), param_grid=param_grid, cv=3, n_jobs=-1)

# # Perform the grid search
# grid_search_result = grid_search.fit(X_train_reshaped, y_train_scaled)

# # Print the best parameters and best score
# print(f"Best Parameters: {grid_search_result.best_params_}")
# print(f"Best Score: {grid_search_result.best_score_}")


In [75]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dropout, Dense, Input
# from scikeras.wrappers import KerasRegressor
# from sklearn.model_selection import GridSearchCV

# # Define the function to create the LSTM model
# def create_lstm_model(units=50, dropout_rate=0.5, optimizer='adam'):
#     model = Sequential()
#     model.add(Input(shape=(timesteps, X_train_reshaped.shape[2])))  # Use Input layer
#     model.add(LSTM(units=units, return_sequences=True))
#     model.add(Dropout(dropout_rate))
#     model.add(LSTM(units=units, return_sequences=False))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(units=1))
#     model.compile(optimizer=optimizer, loss=rmsle)
#     return model

# # Define the grid of hyperparameters to search
# model_grid = [
#     ('model_1', lambda: create_lstm_model(units=50, dropout_rate=0.2, optimizer='adam')),
#     ('model_2', lambda: create_lstm_model(units=60, dropout_rate=0.2, optimizer='adam')),
#     ('model_3', lambda: create_lstm_model(units=80, dropout_rate=0.2, optimizer='adam')),
#     ('model_4', lambda: create_lstm_model(units=50, dropout_rate=0.3, optimizer='adam')),
#     ('model_5', lambda: create_lstm_model(units=60, dropout_rate=0.3, optimizer='adam')),
#     ('model_6', lambda: create_lstm_model(units=80, dropout_rate=0.3, optimizer='adam')),
#     ('model_7', lambda: create_lstm_model(units=50, dropout_rate=0.5, optimizer='adam')),
#     ('model_8', lambda: create_lstm_model(units=60, dropout_rate=0.5, optimizer='adam')),
#     ('model_9', lambda: create_lstm_model(units=80, dropout_rate=0.5, optimizer='adam')),
#     ('model_10', lambda: create_lstm_model(units=50, dropout_rate=0.2, optimizer='rmsprop')),
#     ('model_11', lambda: create_lstm_model(units=60, dropout_rate=0.2, optimizer='rmsprop')),
#     ('model_12', lambda: create_lstm_model(units=80, dropout_rate=0.2, optimizer='rmsprop')),
#     ('model_13', lambda: create_lstm_model(units=50, dropout_rate=0.3, optimizer='rmsprop')),
#     ('model_14', lambda: create_lstm_model(units=60, dropout_rate=0.3, optimizer='rmsprop')),
#     ('model_15', lambda: create_lstm_model(units=80, dropout_rate=0.3, optimizer='rmsprop')),
#     ('model_16', lambda: create_lstm_model(units=50, dropout_rate=0.5, optimizer='rmsprop')),
#     ('model_17', lambda: create_lstm_model(units=60, dropout_rate=0.5, optimizer='rmsprop')),
#     ('model_18', lambda: create_lstm_model(units=80, dropout_rate=0.5, optimizer='rmsprop'))
# ]

# param_grid = {
#     'model': [m[1] for m in model_grid],
#     'batch_size': [16, 32, 64],  # Batch size
#     'epochs': [30, 50, 100],  # Number of epochs
# }

# # Create a GridSearchCV object
# grid_search = GridSearchCV(estimator=KerasRegressor(model=create_lstm_model, verbose=0), param_grid=param_grid, cv=3, n_jobs=-1)

# # Perform the grid search
# grid_search_result = grid_search.fit(X_train_reshaped, y_train_scaled)

# # Retrieve the best parameters
# best_params = grid_search_result.best_params_
# best_model_name = model_grid[[m[1] for m in model_grid].index(best_params['model'])][0]

# # Print the best parameters and best score
# print(f"Best Model: {best_model_name}")
# print(f"Best Batch Size: {best_params['batch_size']}")
# print(f"Best Epochs: {best_params['epochs']}")
# print(f"Best Score: {grid_search_result.best_score_}")


Best Model: model_11
Best Batch Size: 64
Best Epochs: 30
Best Score: 0.00651280349452817


In [76]:
# Predictions
lstm_predictions = lstm_model.predict(X_test_reshaped)

# Ensure both predictions and actual values are of type float64
y_test_scaled = y_test_scaled.reshape(-1, 1).astype('float64')
lstm_predictions_scaled = lstm_predictions.astype('float64')

# Calculate RMSLE
lstm_rmsle_val = rmsle(y_test_scaled, lstm_predictions_scaled)

print(f"LSTM RMSLE VAL: {lstm_rmsle_val.numpy()}")


NameError: name 'cgvhbn' is not defined

In [ ]:
val_results_df = pd.DataFrame({
    'Actual': y_test_scaled.flatten(),  # Flatten to ensure the shape is 1D
    'Predicted': lstm_predictions_scaled.flatten()
})

# Display the first few rows of the DataFrame
print(val_results_df.head())

In [ ]:
lstm_predictions_scaled.shape

In [ ]:
y_test_scaled.shape

In [ ]:
y_test_actual = target_scaler.inverse_transform(y_test_scaled).astype('float64')
lstm_predictions_actual = target_scaler.inverse_transform(lstm_predictions_scaled).astype('float64')

In [ ]:
val_results_actual_df = pd.DataFrame({
    'Actual': y_test_actual.flatten(),  # Flatten to ensure the shape is 1D
    'Predicted': lstm_predictions_actual.flatten()
})

# Display the first few rows of the DataFrame
print(val_results_actual_df.head())

In [ ]:
lstm_rmsle_val_actual = rmsle(y_test_actual, lstm_predictions_actual)
lstm_rmsle_val_actual

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(y_test_actual, label='Actual Values', color='b')
plt.plot(lstm_predictions_actual, label='Predicted Values', color='r')
plt.xlabel('Days')
plt.ylabel('Predicted vs Actual')
plt.title('Actual vs Predicted Sales')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(y_test_actual, lstm_predictions_actual, alpha=0.5)
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title('Actual vs Predicted Sales')
plt.plot([min(y_test_actual), max(y_test_actual)], [min(y_test_actual), max(y_test_actual)], color='red')  # Diagonal line
plt.show()


## predcition on test.csv

In [ ]:
predictions_list=[]

In [ ]:
test_df = pd.read_csv(mydir + 'preprocessed_test_data.csv')
test_df.head()

In [ ]:
# Step 1: Group by 'store_nbr' and 'family'
grouped_test = test_df.groupby(['store_nbr', 'family'])

# Step 2: Sort each group by 'date' in ascending order
sorted_test_df = grouped_test.apply(lambda x: x.sort_values('date')).reset_index(drop=True)

# Optional: If you want to create a dictionary of DataFrames, one for each group
grouped_test_dict = {f"test_{store}_{family}": group.sort_values('date') 
                for (store, family), group in grouped_test}

# Display the result
sorted_test_df.head()


In [ ]:
test_1_AUTOMOTIVE_df = grouped_test_dict['test_1_BEVERAGES']
test_1_AUTOMOTIVE_df.head()

In [ ]:
test_1_AUTOMOTIVE_df.columns

In [ ]:
test_1_AUTOMOTIVE_df.shape

In [ ]:
# Create cyclical features
test_1_AUTOMOTIVE_df['month_sin'] = np.sin(2 * np.pi * test_1_AUTOMOTIVE_df['month'] / 12)
test_1_AUTOMOTIVE_df['month_cos'] = np.cos(2 * np.pi * test_1_AUTOMOTIVE_df['month'] / 12)
test_1_AUTOMOTIVE_df['weekday_sin'] = np.sin(2 * np.pi * test_1_AUTOMOTIVE_df['week_of_day'] / 7)
test_1_AUTOMOTIVE_df['weekday_cos'] = np.cos(2 * np.pi * test_1_AUTOMOTIVE_df['week_of_day'] / 7)

# Optionally, create lag features and rolling statistics
# processed_1_AUTOMOTIVE_df['sales_lag_1'] = processed_1_AUTOMOTIVE_df['sales'].shift(1)
# processed_1_AUTOMOTIVE_df['sales_rolling_mean_7'] = processed_1_AUTOMOTIVE_df['sales'].rolling(window=7).mean()

# Drop rows with NaN values if needed
test_1_AUTOMOTIVE_df = test_1_AUTOMOTIVE_df.dropna()

test_1_AUTOMOTIVE_df.shape

In [ ]:
test_1_AUTOMOTIVE_df_cp = test_1_AUTOMOTIVE_df.copy()

In [ ]:
# drop unnecessary columns 
test_1_AUTOMOTIVE_df_cp = test_1_AUTOMOTIVE_df_cp.drop([
    'Unnamed: 0',
    'date',
    'id',
    'city',
    'state',
   'store_nbr',
     'family',
       'type' ], 
       axis = 1)

In [ ]:
test_1_AUTOMOTIVE_df_cp.columns

In [ ]:
test_1_AUTOMOTIVE_df_cp_scaled = feature_scaler.fit_transform(test_1_AUTOMOTIVE_df_cp)

In [ ]:
test_reshaped = test_1_AUTOMOTIVE_df_cp_scaled.reshape((test_1_AUTOMOTIVE_df_cp_scaled.shape[0], 
                                                               timesteps, 
                                                               test_1_AUTOMOTIVE_df_cp_scaled.shape[1]))

# Make predictions using the LSTM model
lstm_predictions_test_normalised = lstm_model.predict(test_reshaped)

In [ ]:
lstm_predictions_test_normalised

In [ ]:
lstm_predictions_test_normalised_reshaped = lstm_predictions_test_normalised.astype('float64')
lstm_predictions_test_actual = target_scaler.inverse_transform(lstm_predictions_test_normalised_reshaped).astype('float64')

In [ ]:
lstm_predictions_test_actual

In [ ]:
predictions_df = pd.DataFrame({
        'id': test_1_AUTOMOTIVE_df['id'],
        'store_nbr': 1,
        'family': 'BEVERAGES',
        'date': test_1_AUTOMOTIVE_df['date'],  # Or any other identifier column you have
        'predictions': lstm_predictions_test_actual.flatten()  # Ensure predictions is 1D
    })

In [ ]:
predictions_list.append(predictions_df)

print(predictions_df)